# MovieLens: A Basic Regression Model

This notebook explores a basic regression model dubbed "the baseline model" to predict user-movie ratings on the MovieLens 25M dataset. We use the [Surprise Library](https://surprise.readthedocs.io/en/stable/index.html) to load and split data, as well as fit models. The theory behind the procedures follows [Koren, Y. - Factor in the Neighbors: Scalable and
Accurate Collaborative Filtering](https://courses.ischool.berkeley.edu/i290-dm/s11/SECURE/a1-koren.pdf).

# Setup

In [2]:
import numpy as np
import pandas as pdb
import surprise

import sys
sys.path.insert(1, "..")

from source_code.Python import get_data

In [3]:
df_ratings = get_data.get_movielens_large_ratings()
df_movies = get_data.get_movielens_large_movies()

In [5]:
df_ratings.head()

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510


In [10]:
df_ratings.dtypes

userId         int64
movieId        int64
rating       float64
timestamp      int64
dtype: object

# The Surprise Library

The `surprise` library is a scikit learn style package that allows for constructions of well-known recommender systems. For this notebook, we will be exploring a basic linear regression-style model to predict user-movie ratings.

## Readers and Datasets

In order to use the built-in functions of `surprise`, we must first construct a framework for how to prepare and ingest our data set. The two main components of the `surprise` library that helps us do this are:

1) `Reader` objects. These object store metadata for how to interpret our data set.
2) The `Dataset` module which contains functions that load/stream data from the source. The source can either be a file on a disk (e.g. a CSV file) or a pandas dataframe.

The idea is to stream rows of data using the `Dataset` functions into memory, while using the `Reader` object to understand that data stream.

In [11]:
# Reader and Dataset for a csv file;
# Loading data from a file requires the load_from_file() method;
# NOTE: if the csv has a header (ie column names) we have to use the "skip_lines" parameter
file_path = "../data/movielens/large/ratings.csv"

reader_csv = surprise.Reader(line_format = "user item rating timestamp", sep = ",", skip_lines = 1)
dataset_csv = surprise.Dataset.load_from_file(file_path, reader = reader_csv)

# Reader and Dataset on an existing pandas dataframe;
# loading data from a dataframe requires the load_from_df() method
reader_df = surprise.Reader(rating_scale = (1, 5))
dataset_df = surprise.Dataset.load_from_df(df_ratings[["userId", "movieId", "rating"]], reader = reader_df)

# Baseline Predictions

Let us begin from the ground up by asking the simple question: "What do we think user $u$ will rate movie $j$?" 

If the user has already rated the movie, than no prediction is necessary on other part. The challenge comes for movies that user $i$ has not yet rated. For such missing ratings, we have to impute the values and thus the problem is fundamentally a problem of regression.

## Ratings Data: Pooling All Users and Movies

Let's start at the most atomic level and suppose all we had was a list `ratings = [1.0, 3.5, 2.5, ...]`. In other words, we treat all users and all movies as the same and think about a giant pool of ratings. 

The most simple approach is to use the **average** across all movies by all users. Recall that the (arithmetic) average $\mu$ is the "constant of best fit" in the sense that $\mu$ is the constant which minimizes the mean squared error. Absent any other information, this numerical value $\mu$ would be our best-guess at what a missing rating is most likely to be.

In [14]:
r_bar = df_ratings[['rating']].mean()

r_bar

rating    3.533854
dtype: float64

In the MovieLens data set, our $\mu$ has a value of 3.53, so this gives us an initial guess at what the missing ratings might be.

$$r_{uj} := \mu = 3.53$$

for all users $u$ and movies $j$.

## Item Bias: Accounting For Good and Bad Movies

The simplistic approach above only usings the `ratings` data but not the movie ID data. Specifically, the simplistic approach ignores the fact that the ratings belong to different movies. This is problematic because the movie ratings belong to clusters based on the movie being rated. For example:

- All the ratings with `movieId` equal to 1 belong to Toy Story, which is generally regarded as a very good film. Therefore, we would expect all ratings with `movieId == 1` to generally be higher than an average movie. 
- Conversely, all the ratings with `movieId` equal to 74754 belong to The Room (starring Tommy Wiseau), which is generally regarded as a terrible film. Therefore, we would expect all such ratings to be generally lower than an average movie.


In [28]:
df_movies[(df_movies["movieId"] == 1) | (df_movies["movieId"] == 74754)]

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
14391,74754,"Room, The (2003)",Comedy|Drama|Romance


In [35]:
# average rating for Toy Story
df_ratings[df_ratings["movieId"] == 1][["rating"]].mean()

rating    3.893708
dtype: float64

In [36]:
# average rating for The Room
df_ratings[df_ratings["movieId"] == 74754][["rating"]].mean()

rating    2.403731
dtype: float64

As expected:

- Toy Story has an average rating of 3.89 which is +0.36 higher than the global average of 3.53
- The Room has an average rating of 2.40 which is -1.13 lower than the global average of 3.53

So how do we incorprate this information into our prediction? What we can do is take our simplistic global average of 3.53 and adjust it based on how much each individual movie is above or below the global average.

$$r_{uj} := \mu + b_j = 3.53 + b_j$$

where $b_j$ is the difference between movie $j$'s average rating and the global average $\mu$. Note that this is equivalent to saying $r_{uj} = \text{average rating for movie }j$

So for example, if we wanted to predict user $u$'s rating for Toy Story, it would just be:

$$r_{u1} = \mu + b_1 = 3.53 + 0.36 = 3.89$$

Similarly, if we wanted to predict user $u$'s rating for The Room, it would just be:

$$r_{u,74754} = \mu + b_{74754} = 3.53 - 1.13 = 2.40$$

## User Bias: Accounting for Harsh and Lenient Reviewers

In the same way that movies can be above or below average, a user can be more harsh or lenient when it comes to movie reviews. Some users generally rate movies loweer than the average person, while some users generally rate movies higher than the average person. A good way to think about this is to recognize that different people have different grading scales: some people think that a 3-star rating represents an average movie, while other people think that a 3-star rating represents a below average movie.

We can account for this type of information in exactly the same way as before: by considering the average value across all ratings given out by a user.

$$r_{uj} = \mu + b_u + b_j = 3.53 + b_u + b_j$$

where $b_u$ is the difference between user $u$'s average rating and the global average.

## A Regression Model

We can unify all 3 considerations together and derive a baseline model that predicts user ratings:

$$\hat{r}_{uj} = \mu + b_u + b_j$$

In the discussion above, the parameters $b_u$ and $b_j$ are known by taking the averages of the ratings by user and by movie.

A more sophisticated approach is to treat this as a statistical model by allowing for sampling variance in the data set. More precisely, $b_u$ should represent a user $u$'s average rating across *all* movies in the data set. Similarly, $b_j$ should represent movie $j$'s average rating across *every* person in the data set.

From this perspective, the movie ratings in our data set are just a sample from larger population; we only observe a few of the possible user-movie ratings. The values of $\mu$, $b_u$, and $b_j$ will almost certainly fluctuate if we had observed a different combination of user-movie ratings. This fluctation is *sampling variance* and we want to account for this uncertainty in our model.

We can thus approach this as a linear regression problem. Our hypothesized probability model is:

$$r_{uj} = \mu + \sum_{u = 1}^{U} b_i\cdot I(i) + \sum_{j = 1}^{M} b_j \cdot I(j) + \epsilon$$


where $I(i)$ and $I(j)$ are indicator variables for user $i$ and movie $j$ and $\epsilon \sim N(0, \sigma)$. From this probability model, it is possible to calculate a *likelihood* for each possible value of $\mu$, $b_i$, and $b_j$ based on the data we are currently observing:

$$\mathcal{L}(\mu, b_i, b_j | X) = Pr(X | \mu, b_i, b_j)$$

Our "best guess" estimates for $\mu$, $b_i$, are $b_j$ the values of $\mu$, $b_i$, $b_j$ which maximizise this likelihood:

$$\mu, b_i, b_j = \max_{\mu, b_i, b_j} \mathcal{L}(\mu, b_i, b_j | X)$$

As it turns out, the solution to this maximization problem is precisely the same as the solution to that minimizes the following loss function:

$$min \sum_{(u, j) \in R} (r_{uj} - \mu - b_u - b_i)^2$$

Finally, we can also stipulate some Bayesian priors $b_i \sim N(0, \lambda_1)$ and $b_j \sim N(0, \lambda_2)$ and the result is the following minimization problem:

$$min \sum_{(u, j) \in R} (r_{uj} - \mu - b_u - b_i)^2 + \lambda_1\sum_{i} b_i^2 + \lambda_2\sum_{j} b_j^2$$

## Baseline Model with Surprise

The ideas discussed above can be achieved using the `BaselineOnly` model in `surprise`

In [39]:
# instantiate a Baseline Model
baseline_model = surprise.BaselineOnly()

# perform 5-fold cross validation to evaluate model specification
surprise.model_selection.cross_validate(baseline_model, dataset_df, measures = ["RMSE", "MAE"], cv = 5, verbose = True)

Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Evaluating RMSE, MAE of algorithm BaselineOnly on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8602  0.8594  0.8597  0.8599  0.8597  0.8598  0.0003  
MAE (testset)     0.6566  0.6561  0.6563  0.6563  0.6562  0.6563  0.0002  
Fit time          86.41   93.15   93.18   95.98   94.42   92.63   3.28    
Test time         47.17   46.57   40.85   48.71   41.37   44.94   3.20    


{'test_rmse': array([0.86019934, 0.85942653, 0.8596957 , 0.85988404, 0.85967176]),
 'test_mae': array([0.65660484, 0.65611804, 0.65627582, 0.65629546, 0.65619791]),
 'fit_time': (86.40905952453613,
  93.15286207199097,
  93.17840123176575,
  95.97624039649963,
  94.42246890068054),
 'test_time': (47.1684045791626,
  46.573326587677,
  40.85033297538757,
  48.71293568611145,
  41.37364172935486)}

The baseline model has a mean MSE of 0.86 across the 5 folds with a mean MAE of 0.66. This means on average, the baseline model deviates by more little more than half-a-star.

Notice in the console output, the biases were estimated using ALS or **Alternating Least Squares**. This procedure starts by randomly assigning values to $b_i$ and minimizing with respect to $b_u$ first, while holding $b_i$ fixed. Once the $b_u$ are estimated, the procedure minimizes with respect to $b_i$ holding $b_u$ fixed. Then we return and re-minize to $b_u$ and so on, alternating between minimizing one the sets $b_i$ and $b_u$, while holding the other fixed.

We can also do a more rudimentary train-test split: fit the model on the training set, and evaluate the performance on the test set.

In [41]:
# train-test split with 30% going to the test set
trainset, testset = surprise.model_selection.train_test_split(dataset_df, test_size = 0.3)

In [44]:
# fit baseline model on training set
baseline_model.fit(trainset)

Estimating biases using als...


In [45]:
# predict on test set and evaluate performance
predictions = baseline_model.test(testset)

surprise.accuracy.rmse(predictions)

RMSE: 0.8602


0.8602156904794773

In [48]:
# make a prediction on a user-movie rating
# note that userId and movieId must be passed as a string
baseline_model.predict(uid = "1", iid = "74754", verbose = True)

user: 1          item: 74754      r_ui = None   est = 3.53   {'was_impossible': False}


Prediction(uid='1', iid='74754', r_ui=None, est=3.5335277878380573, details={'was_impossible': False})

The baseline model predicts that User 1 will give Movie 74754 (The Room) a rating of 3.5, implying that User 1 would rate The Room higher than an average user.

In [50]:
# check User 1's average rating given out
df_ratings[df_ratings["userId"] == 1].mean()

userId       1.000000e+00
movieId      6.740814e+03
rating       3.814286e+00
timestamp    1.147873e+09
dtype: float64

We see User 1 generally gives out an average rating of 3.8, so a rating of 3.5 is actually below average for User 1. 

We can also play around with hyperparameters and options for the baseline model; this is done by passing a dictionary. Note that the regularization parameters were chosen based on [Koren 2010] which mentions that `reg_u = 25` and `reg_i = 10` were found to work the best on the Netflix Prize dataset.

In [51]:
# set hyperparameters and estimation method
baseline_options = {
    "method": "als",  # estimation method
    "n_epochs": 5,    # number of iterations of ALS procedure; default is 10
    "reg_u": 25,      # regularization parameter for user biases
    "reg_i": 10       # regularization parameter for movie biases
}

baseline_model = surprise.BaselineOnly(bsl_options = baseline_options)

In [52]:
# fit and test
baseline_model.fit(trainset)

predictions = baseline_model.test(testset)

surprise.accuracy.rmse(predictions)

Estimating biases using als...
RMSE: 0.8622


0.862169563962609

Since we have hyperparameters we can tune, we can further attempt to optimize the model by using a grid search to select optimal hyperparameters. Actual training time will become rather long on the full 25 million row dataset, so we switch over to the 100,000 row dataset.

In [63]:
# load smaller dataset
df_ratings = get_data.get_movielens_small_ratings()

# setup Reader obejct and Dataset loader
reader_df = surprise.Reader(rating_scale = (1, 5))
dataset_df = surprise.Dataset.load_from_df(df_ratings[["userId", "movieId", "rating"]], reader = reader_df)

# setup param_grid that we search over
# Note: for the BaselineOnly model, the hyperparameters are passed
# as a dictionary to the bsl_options argument. 
# This means our param_grid will have to be finessed like so:
param_grid = {
    "bsl_options" : {
        "reg_u" : [10, 25],
        "reg_i" : [10, 25]
    }
}

# instantiate GridSearchCV object
gs = surprise.model_selection.GridSearchCV(
    surprise.BaselineOnly, 
    param_grid, 
    measures = ["rmse", "mae"], 
    cv = 3
)

# conduct grid search
gs.fit(dataset_df)

# best RMSE score
print(gs.best_score["rmse"])

# best MAE score
print(gs.best_score["mae"])

# return the best configuration wrt to rmse
print(gs.best_params["rmse"])

Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
0.8763149214791635
0.6758180562256308
{'bsl_options': {'reg_u': 10, 'reg_i': 10}}


# Limitations and Considerations

The baseline model is simplistic and a "natural" first choice, but it's simplicity comes at the cost of limitations.

- The model is only able to make predictions on users and movies that exist in the training set. In particular, if a new user joins and rates a few movies, we would need to refit in order to make predictions on this user's ratings.
- More generally, if a new user joins the data set, the model cannot make predictions on their ratings *until* we learn about the new user's preferences. This is a fundamental and universal problem in recommender systems called the **cold start problem**.
- Our model does not take into account similarities between users' rating behaviors. Specifically, if User 1 and User 2 both rate a common set of movies very highly, it is likely that User 1's ratings can be used to predict User 2's ratings (and vice versa). This idea of using common rating behaviors across users is called **user-based collaborative filtering**.
- Similarly, our model does not take into account similarities between movies' rating behaviors. If Movie 1 and Movie 2 are both rated highly by a common group of users, then it is likely that other movies rated highly by a user in the group will be rated highly by all users in the group. This idea of using common rating patterns across movies is called **item-baesd collaborative filtering**.
- Our model does not incorporate any additional pieces of information about the movies such as genre, tags, and text-based reviews. For example, if User 1 rated The Terminator, The Matrix, and Star Wars very highly, then it is likely User 1 will rate Sci-Fi Action movies very highly in general. This idea of using implicit characteristics of movies is called **content-based recommendation**.
- Additionally, the model cannot learn to recognize which movies are similar. If a group of users rates The Terminator and The Matrix very similarly, then we should be able to infer that these two movies must have *something* in common (i.e. they have the same "vibe") This idea of finding implicit characteristics between movies is called **latent factor modeling**.